Moodle XML Question Generator, (C) 2019-2020 Kosaku Nagasaka (Kobe University)

# moodle_xqgの使い方

Moodle XML Question Generator の Python 版である moodle_xqg の基本的な使い方について説明しています。

## パッケージの読み込み

**moodle_xqg.core**は必ず必要です。**moodle_xqgb.qbank.common**は数式のLaTeX化など，問題作成に必要なものが含まれているので，読み込んだほうがよいです。

In [1]:
import moodle_xqg.core as mxqg
import moodle_xqg.qbank.common as mxqg_common

次のパッケージは作られる問題に依存しますが，多くの場合，乱数生成と数式処理のために，必要となると思います。

In [2]:
import random
import sympy

## 最小限の問題サンプル

問題生成器は，**Question**の小クラスとして定義する必要があります。必要なメソッドは，**question_generate**と**correct_answers_generate**と**incorrect_answers_generate**と**question_text**と**answer_text**の5つです。

問題生成器は，**Quiz.quiz_identifier**に同じ問題であるかを判定可能なデータを必ず格納してください。この値でもって問題の区別をするので，格納していない場合，最初の1問しか自動生成されなくなってしまいます。 

In [3]:
class integer_addition_simple(mxqg.Question):
    def question_generate(self, _quiz_number=0):
        quiz = mxqg.Quiz(name='整数の足し算', quiz_number=_quiz_number, lang='ja')
        # 問題データの生成
        quiz.data = [random.choice([-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9]) for i in range(2)]
        quiz.quiz_identifier = hash('{}+{}'.format(quiz.data[0], quiz.data[1]))
        # 正答の生成
        quiz.data.append(quiz.data[0] + quiz.data[1])
        # 正答の選択肢の生成
        ans = { 'fraction': 100, 'data': quiz.data[2] }
        quiz.answers.append(ans)
        return quiz        
    def correct_answers_generate(self, quiz, size=1):
        # 正答を個別には作らないので何もしない
        pass
    def incorrect_answers_generate(self, quiz, size=4):
        # ランダムに誤答を作る（本来は推奨されない）
        answers = []
        ans = { 'fraction': 0, 'data': quiz.data[2] }
        while len(answers) < size:
            ans['data'] += random.randint(-9,9)
            ans['feedback'] = '丁寧に計算し直してみましょう。'
            if ans['data'] != quiz.data[2]:
                answers.append(dict(ans)) # 同じ ans を使いまわすときはコピーが必要
            answers = mxqg_common.answer_union(answers)
        return answers    
    def question_text(self, quiz):
        return '次の数式の値を求めよ。<br />' + \
                mxqg_common.sympy_str_to_text('{}+{}'.format(quiz.data[0], quiz.data[1]))
    def answer_text(self, ans):
        return mxqg_common.sympy_expr_to_text(ans['data'])

問題の自動生成は，**generate**を使いますが，それに先立ち，問題生成器のインスタンスを生成しておく必要があります。作成する問題数は，*generate*に**size=100**のような形で指定します（無指定時は，10個生成されます）。

In [4]:
ias = integer_addition_simple()
ias_quizzes = mxqg.generate(ias, category='最小限の問題サンプル')

生成結果は，**Quizzes**のインスタンスとなっており，次のようにプレビューができます。

In [5]:
ias_quizzes.listview()

In [6]:
ias_quizzes.preview(size=2)

語句の修正をする場合は，辞書型で指定します。次の例では「数式」をすべて「計算結果」に置き換えた状態でプレビューが行われます。ただし，インスタンスの中身自体は変更されていないので注意をしてください。

In [13]:
ias_quizzes.preview(size=2, translate={'数式':'計算結果'})

問題整理番号を本文に記載せず，問題表題にのみ付与する場合は次のようにオプションを付けます。

In [7]:
ias_quizzes.preview(size=2, show_quiz_number_adminonly=True)

このようにして生成した結果は，**save**を使ってMoodleにインポート可能なファイルに保存することができます。

In [7]:
ias_quizzes.save('integer_addition_simple_in_japanese.xml', translate={'数式':'計算結果'})

実際にインポートして確認してみてください。なお，問題整理番号の表示が不要な場合は，**show_quiz_number=False**をオプションとして*save*に付けてください。

## 最小限の問題サンプルの多少の改善

次にもう少しだけ複雑な問題生成器を作成します。前述のものを少しだけ修正しています。

In [8]:
class integer_addition(mxqg.Question):
    def __init__(self, lower=-10, upper=10):
        # 生成する整数の範囲を指定できるように修正
        self.lower = lower
        self.upper = upper
    def question_generate(self, _quiz_number=0):
        quiz = mxqg.Quiz(name='整数の足し算', quiz_number=_quiz_number, lang='ja')
        # 問題データの生成（SymPyは，0を計算せずにLaTeXへは変換できないので，0を除外してる）
        quiz.data = [random.choice(list(set(range(self.lower, self.upper + 1))-{0})) for i in range(2)]
        quiz.quiz_identifier = hash('{}+{}'.format(quiz.data[0], quiz.data[1]))
        # 正答の生成
        quiz.data.append(quiz.data[0] + quiz.data[1])  
        # 正答の選択肢の生成
        ans = { 'fraction': 100, 'data': quiz.data[2] }
        quiz.answers.append(ans)
        return quiz        
    def correct_answers_generate(self, quiz, size=1):
        # 正答を個別には作らないので何もしない
        pass
    def incorrect_answers_generate(self, quiz, size=4):
        answers = []
        ans = { 'fraction': 0 }
        # 二項演算子の勘違い
        ans['data'] = quiz.data[0] - quiz.data[1]
        ans['feedback'] = '引き算ではありません。'
        if ans['data'] != quiz.data[2]:
            answers.append(dict(ans))
        ans['data'] = quiz.data[0] * quiz.data[1]
        ans['feedback'] = '掛け算ではありません。'
        if ans['data'] != quiz.data[2]:
            answers.append(dict(ans))
        if quiz.data[1] != 0:
            ans['data'] = sympy.Rational(quiz.data[0], quiz.data[1])
            ans['feedback'] = '割り算ではありません。'
            if ans['data'] != quiz.data[2]:
                answers.append(dict(ans))
        answers = mxqg_common.answer_union(answers)
        if len(answers) >= size:
            return random.sample(answers,k=size)
        # ランダムに誤答を作る（本来は推奨されない）
        count = 0
        ans['data'] = quiz.data[0] + quiz.data[1]
        while len(answers) < size and count < 10:
            count += 1
            ans['data'] += random.randint(self.lower, self.upper)
            ans['feedback'] = '丁寧に計算し直してみましょう。'
            if ans['data'] != quiz.data[2]:
                answers.append(dict(ans))
            answers = mxqg_common.answer_union(answers)
        return answers    
    def question_text(self, quiz):
        return '次の数式の値を求めよ。<br />' + \
                mxqg_common.sympy_str_to_text('{}+{}'.format(quiz.data[0], quiz.data[1]))
    def answer_text(self, ans):
        return mxqg_common.sympy_expr_to_text(ans['data'])

問題生成器が引数を取れるのであれば，インスタンス生成時に与えるようにします。

In [14]:
ia = integer_addition(lower=-500, upper=500)
ia_quizzes = mxqg.generate(ia, category='最小限の問題サンプルの多少の改善')

なお，プレビュー時には，フィードバックや割り当てられた得点などが，マウスを選択肢などに重ねた際に，ポップアップ表示されるようになっているので，確認してみてください。

In [15]:
ia_quizzes.preview(size=2)

ファイルへの出力は，XML形式だけではなく，**internal=True**を指定することで，オブジェクトのまま保存することも可能です。

In [17]:
ia_quizzes.save('integer_addition_in_japanese.pickle', internal=True)

この形式で保存したものは，あとで作業のために再度読み込めます。読み込みには，**load_internal**を使ってください。

In [18]:
ia_quizzes = mxqg.load_internal('integer_addition_in_japanese.pickle')

In [19]:
ia_quizzes.preview(size=1)

なお，一旦，XML形式に変換したものであっても，再度読み込むことは可能です。

In [21]:
ia_quizzes.save('integer_addition_in_japanese.xml')

XML形式からの読み込みには，**load_xml**を使います。ただし，XML形式からの読み込み時には，問題生成時には存在した問題作成用の内部データ（*data*）や本来の問題整理番号（*quiz_number*）や区別子（*quiz_identifier*）は存在しないので，少しだけ注意が必要です。

In [22]:
ia_quizzes = mxqg.load_xml('integer_addition_in_japanese.xml')

具体的には，*preview*やさらなる*save*のときには，**show_quiz_number=False**を付けてあげる必要があります。

In [23]:
ia_quizzes.preview(size=1, show_quiz_number=False)